In [1]:
import ibis

In [2]:
ibis.options.interactive = True

In [5]:
connection = ibis.sqlite.connect('club/club.db')

In [6]:
connection.list_tables()

['attends',
 'budgets',
 'events',
 'expenses',
 'fees',
 'has',
 'incurs',
 'majors',
 'members',
 'pays',
 'supports',
 'zipcodes']